# Introduction to Collaborative Filtering

### Example extracted from *Programming Collective Intelligence* by Toby Segaran.

#### 1 - Collecting and storing movie ratings

Below, we define a dictionary of movie critics and their ratings of a small set of movies:

In [1]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

We can then easily pull data from the above dictionary of ratings. For example, if we wish to know what score Lisa Rose gave to the movie *Lady in the Water*, we can simply use the following:

In [2]:
critics['Toby']['Snakes on a Plane']

4.5

Likewise, if we wish to see all ratings provided by Toby:

In [3]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

#### 2 - Finding Similar Users

After collecting the ratings, we now need to determine how similar people are in their tastes. We will provide two approaches for doing so:  Euclidean distance and Pearson correlation.

Euclidean distance scores can be illustrated by taking the items that people ranked in common and using them as axes for a chart as seen below:

In [4]:
from IPython.core.display import Image 
Image(width=600,url='http://i.imgur.com/TeM7mJs.png')

The closer people are in the above chart, the more similar are their preferences. Note, however, that a significant limitation of this approach is that since it uses a two-dimentional chart, we can only compare people based on their ratings of two movies. However, the same principle can be expanded to higher dimentions.

We can compute the distance between Toby and LaSalle above by using the Euclidean distance formula:

<h3 align="center">$$d(t,l) = \sqrt{(t_1-l_1)^2 + (t_2-l_2)^2}$$</h3> 

In [5]:
from math import sqrt
sqrt(pow(5-4,2)+pow(4-1,2))

3.1622776601683795

The above result gives us the actual distance between each person. If we wish to have a score that assumes higher values for people who are more similar, we can add 1 to the formula and invert it. All values will now be between 0-1.

In [6]:
1/(1+sqrt(pow(5-4,2)+pow(4-1,2)))

0.2402530733520421

Putting everything together into a function for calculating similarity gives:

In [7]:
# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    
    # Get the list of shared_items
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
    
    # if they have no ratings in common, return 0
    if len(si)==0: return 0
    
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
                        for item in prefs[person1] if item in prefs[person2]])

    return 1/(1+sum_of_squares)

The following gives us the similarity score between Lisa Rose and Gene Seymour:

In [8]:
sim_distance(critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

We can also easily check if Lisa is more similar to Gene or Toby as follows:

In [9]:
s1 = sim_distance(critics,'Lisa Rose','Gene Seymour'); s2 = sim_distance(critics,'Lisa Rose','Toby')
if s1 > s2:
    print 'Lisa is more similar to Gene than she is to Toby. Similarity score = ' + str(s1)
else:
    print 'Lisa is more similar to Toby than she is to Gene. Similarity score = ' + str(s2)

Lisa is more similar to Toby than she is to Gene. Similarity score = 0.222222222222


Further, we can compute similarities using a more sophisticated approach. Pearson correlation coefficients measure how well two sets of data fit on a straight (*best-fit*) line. We can illustrate this by plotting all ratings by two different people on a chart. The closer these ratings are to the line, the more similar these individuals are. Below is an example of such plots:

In [10]:
Image(width=600,url='http://i.imgur.com/hzUlTAR.png')

A perfect correlation score of 1 would be illustrated with all ratings falling exactly over the best-fit line. Further, note that one significant advantage of this approach over the previous one is that it corrects for *grade inflation*. That is, if one critic consistently grades movies more harshly than another, as long as the two people follow the same pattern of rating, their correlation scores will still illustrate that similarity, whereas Euclidean distance scores would not.

The following code computes the correlation score between the ratings given by two people. Unlike the distance metric, this formula is not very intuitive, but it does tell you how much the variables change together divided by the product of how much
they vary individually.

In [11]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1

    # Find the number of elements
    n=len(si)

    # if they are no ratings in common, return 0
    if n==0: return 0

    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])

    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])

    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])

    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0

    r=num/den
    return r

Below is a simplified function that computes the exact same thing but uses SciPy to calculate Pearson correlation values:

In [12]:
from scipy.stats.stats import pearsonr
def sim_pearson_fancy(prefs,p1,p2):
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1

    if len(si)==0: return 0
    else:
        si_1 = [prefs[p1][it] for it in si]
        si_2 = [prefs[p2][it] for it in si]
        return pearsonr(si_1, si_2)[0]

This function will return a value between –1 and 1. A value of 1 means that the two people have exactly the same ratings for every item. The following gives us the correlation illustrated by the image above:


In [13]:
sim_pearson(critics,'Lisa Rose','Jack Matthews')


0.7470178808339965

In [14]:
sim_pearson_fancy(critics,'Lisa Rose','Jack Matthews')

0.74701788083399601

#### 3 - Ranking the Critics

Using the above functions, we can write another function that compares everyone against a certain person. This would, for example, allow me to find out which critic is most similar me so that I know whose advice I should take when deciding on a movie.

In [15]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other)
    for other in prefs if other!=person]

    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

Let's see which three critics are most similar to Toby:

In [16]:
topMatches(critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

#### 4 - Recommending Items

Suppose we would like to see movie recommendations instead of comparing ourselves to critics. A naive approach would be to find critics with similar tastes and pick movies that they rated highly and that we have not yet seen. This would, however, cause two problems: (1) it could accidently turn up reviewers that have not reviwed movies that we might like, or (2) it could turn up reviewers that give high ratings to movies that everyone else rates poorly.

To solve these issues, you need to score the items by producing a weighted score that ranks the critics. To do this, you can take the votes of all the other critics and multiply how similar they are to you by the score they gave each movie. The following table shows how this process works when creating recommendations for Toby.


In [17]:
Image(width=800,url='http://i.imgur.com/OLfRWHy.png?1')

Following is a code snippet for a function that returns ranked recommendations for a given user. Both Euclidean distances and Pearson correlation scores can be used.

In [18]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}

    for other in prefs:
        # don't compare me to myself
        if other==person: continue
        sim=similarity(prefs,person,other)

        # ignore scores of zero or lower
        if sim<=0: continue
        for item in prefs[other]:

            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
    
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]

    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

Now you can find out what movies Toby should watch next:

In [19]:
getRecommendations(critics,'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

In [20]:
getRecommendations(critics,'Toby', similarity=sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]